In [19]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from sklearn.preprocessing import LabelEncoder
import scipy.stats as stats
from sklearn.model_selection import train_test_split
import feature_engine.transformation as vt
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.model_selection import RandomizedSearchCV
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor

# the scaler - for min-max scaling
from sklearn.preprocessing import MinMaxScaler

In [20]:
dfnew = pd.read_csv('data/fifa_cleaned.csv')

In [21]:
drop_feature = ['gk_diving','gk_handling','gk_kicking','gk_reflexes','defending_marking','attacking_crossing','attacking_finishing',
'attacking_short_passing','attacking_volleys','defending_median','defending_sliding_tackle',
'defending_standing_tackle','dribbling_median','goalkeeping_diving','goalkeeping_handling','goalkeeping_kicking','goalkeeping_positioning',
'goalkeeping_reflexes','mentality_aggression','mentality_composure','mentality_interceptions','mentality_penalties','mentality_positioning',
'movement_acceleration','movement_agility','movement_balance','movement_sprint_speed','passing_median','physic_median',
'power_long_shots','power_shot_power','power_stamina','shooting_median','skill_curve','skill_fk_accuracy',
'skill_long_passing','wage_eur','weight_kg']

for feature in range(len(drop_feature)):
    dfnew = dfnew.drop([drop_feature[feature]], axis=1)

In [22]:
dfnew.columns

Index(['age', 'height_cm', 'overall', 'potential', 'value_eur',
       'preferred_foot', 'weak_foot', 'skill_moves', 'work_rate',
       'team_position', 'attacking_heading_accuracy', 'skill_dribbling',
       'skill_ball_control', 'movement_reactions', 'power_jumping',
       'power_strength', 'mentality_vision', 'pace_median'],
      dtype='object')

In [23]:
dfnew.to_csv("fifa_final.csv", index=False)

In [24]:
# numerical and categorical features
numerical_features = [feature for feature in dfnew.columns if dfnew[feature].dtype != 'O']
categorical_feature = [feature for feature in dfnew.columns if dfnew[feature].dtype == 'O']

# print columns
print('We have {} numerical features : {}'.format(len(numerical_features), numerical_features))
print('We have {} categorical features : {}'.format(len(categorical_feature), categorical_feature))

We have 16 numerical features : ['age', 'height_cm', 'overall', 'potential', 'value_eur', 'preferred_foot', 'weak_foot', 'skill_moves', 'attacking_heading_accuracy', 'skill_dribbling', 'skill_ball_control', 'movement_reactions', 'power_jumping', 'power_strength', 'mentality_vision', 'pace_median']
We have 2 categorical features : ['work_rate', 'team_position']


In [25]:
# lets move towards splitting the data.

# get the locations
y = dfnew['overall']
X = dfnew.drop(columns=['overall'], axis=1)

In [26]:
# create a column transformer with 3 types of transformer

num_features = X.select_dtypes(exclude="object").columns
cat_features = X.select_dtypes(include="object").columns

from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

numeric_transformer = StandardScaler()
oh_transformer = OneHotEncoder()

preprocessor = ColumnTransformer(
    [
        ("OneHotEncoder", oh_transformer, cat_features),
        ("StandardScaler", numeric_transformer, num_features),
    ]
)

In [ ]:
X = preprocessor.fit_transform(X)

In [ ]:
X.shape

In [ ]:
# split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train.shape, X_test.shape

In [ ]:
def evaluate_model(true, predicted):
    mae = mean_absolute_error(true, predicted)
    mse = mean_squared_error(true, predicted)
    rmse = np.sqrt(mean_squared_error(true, predicted))
    r2_square = r2_score(true, predicted)
    return mae, rmse, r2_square

In [ ]:
models = {
    "Linear Regression" : LinearRegression(),
    "Lasso" : Lasso(),
    "Ridge" : Ridge(),
    "Random Forest Regressor" : RandomForestRegressor(),
    "XGBRegressor" : XGBRegressor(),
    "CatBoosting Regressor" : CatBoostRegressor(verbose = False),
    "AdaBoost Regressor" : AdaBoostRegressor()
}

model_list = []
r2_list = []

for i in range(len(list(models))):
    model = list(models.values())[i]
    model.fit(X_train, y_train)

    # make predictions
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    # Evaluate train and test set
    model_train_mae, model_train_rmse, model_train_r2 = evaluate_model(y_train, y_train_pred)
    model_test_mae, model_test_rmse, model_test_r2 = evaluate_model(y_test, y_test_pred)

    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])

    print("model performance for training dataset")
    print("- Root mean Sqared Error: {:.4f}".format(model_train_rmse))
    print("- mean absolute Error: {:.4f}".format(model_train_mae))
    print("- r2 score: {:.4f}".format(model_train_r2))
    r2_list.append(model_train_r2)

    print("-----------------------------------------------------")

    print("Model performace on test dataset")
    print("- Root mean Sqared Error: {:.4f}".format(model_test_rmse))
    print("- mean absolute Error: {:.4f}".format(model_test_mae))
    print("- r2 score: {:.4f}".format(model_test_r2))  
    r2_list.append(model_test_r2)


    print("="*35)
    print('\n')

In [ ]:
pd.DataFrame(list(zip(model_list, r2_list)), columns=['Model Name','R2_Score']).sort_values(by=['R2_Score'], ascending=False)

In [ ]:
lin_model = LinearRegression(fit_intercept=True)
lin_model = lin_model.fit(X_train, y_train)
y_pred = lin_model.predict(X_test)
score = r2_score(y_test, y_pred)*100
print("Accuracy of the model is %.2f" %score)

In [ ]:
plt.scatter(y_test, y_pred);
plt.xlabel('Actual');
plt.ylabel('Predicted');

In [ ]:
sns.regplot(x=y_test, y=y_pred, ci=None, color='red')

In [ ]:
pred_df = pd.DataFrame({'Actual Value': y_test, 'Predicted Value': y_pred, 'Difference': y_test-y_pred})
pred_df